In [1]:
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', None)

import time
import random

from datetime import datetime, date, timedelta

import os

import random

# Import two files:
* apartments and eviction rates in census blocks.xlsx - output from **Evictions EDA and Geocode**
* ham co apt bldgs by size.csv: you can put this together from the hamilton county auditor's website.
    * go here (https://wedge1.hcauditor.org/) and select "Advanced"
    * Under "Land Use Criteria", download 401, 402, and 403 separately
    * for each csv file, add two columns, "min_size" and "max_size"
    * for each spreadsheet, enter the min and max sizes, as appropriate.  So, for 401, the min_size is 4 and the max_size is 19.  For 403, just enter a very large number for the max_size
    * combine the spreadsheets into one file, and name that file "ham co apt bldgs by size.csv"
* Hamilton County Voter file can be downloaded here: https://www6.ohiosos.gov/ords/f?p=111:1

In [2]:
apt_locations_df = pd.read_excel(r'\apartments and eviction rates in census blocks.xlsx')
apt_sizes_df = pd.read_csv(r'\ham co apt bldgs by size.csv')

In [ ]:
voter_file_df = pd.read_csv(r'C:\Users\kevin\Documents\jupyter notebooks\voter files\Ohio\HAMILTON.txt')

In [4]:
apt_locations_df.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,index_right,Parcel Number,Name,Address,Sale Date,Sale Price,full_address,location,lat,lon,eviction-rate,eviction-filing-rate
0,39,61,4604,1,390610046041,Block Group 1,G5030,S,706969,717,39.10535,-84.386916,"POLYGON ((-84.385125 39.094291, -84.385199 39.094182, -84.385908 39.09413199999999, -84.38604699...",155,003-0005-0110-00,TARPOFF BEVERLY & NICHOLAS,5568 BEECHMONT AVE,5/17/2005,"$202,000","5568 BEECHMONT AVE Hamilton County, Ohio","5568, Beechmont Avenue, Beech-Mar, Cincinnati, Hamilton County, Ohio, 45230-1640, United States ...",39.103828,-84.388522,5.21,8.28
1,39,61,4604,1,390610046041,Block Group 1,G5030,S,706969,717,39.10535,-84.386916,"POLYGON ((-84.385125 39.094291, -84.385199 39.094182, -84.385908 39.09413199999999, -84.38604699...",156,003-0005-0112-00,JWSPIRES LIMITED,5562 BEECHMONT AVE,10/15/2015,"$147,500","5562 BEECHMONT AVE Hamilton County, Ohio","5562, Beechmont Avenue, Beech-Mar, Cincinnati, Hamilton County, Ohio, 45230-1640, United States ...",39.103989,-84.388730,5.21,8.28
2,39,61,4604,1,390610046041,Block Group 1,G5030,S,706969,717,39.10535,-84.386916,"POLYGON ((-84.385125 39.094291, -84.385199 39.094182, -84.385908 39.09413199999999, -84.38604699...",157,003-0005-0115-00,JWSPIRES LIMITED,5556 BEECHMONT AVE,10/15/2015,"$147,500","5556 BEECHMONT AVE Hamilton County, Ohio","5556, Beechmont Avenue, Beech-Mar, Cincinnati, Hamilton County, Ohio, 45230-1640, United States ...",39.104148,-84.389045,5.21,8.28
3,39,61,4604,1,390610046041,Block Group 1,G5030,S,706969,717,39.10535,-84.386916,"POLYGON ((-84.385125 39.094291, -84.385199 39.094182, -84.385908 39.09413199999999, -84.38604699...",14,001-0002-0066-00,SALVADOR PROPERTIES LLC,2215 SALVADOR ST,12/16/2013,"$152,000","2215 SALVADOR ST Hamilton County, Ohio","2215, Salvador Street, Mount Washington Business District, Mount Washington, Cincinnati, Hamilto...",39.094515,-84.386024,5.21,8.28
4,39,61,4604,1,390610046041,Block Group 1,G5030,S,706969,717,39.10535,-84.386916,"POLYGON ((-84.385125 39.094291, -84.385199 39.094182, -84.385908 39.09413199999999, -84.38604699...",24,001-0002-0148-00,VULGUS LLC,2229 SALVADOR ST,8/10/2011,"$175,000","2229 SALVADOR ST Hamilton County, Ohio","2229, Salvador Street, Mount Washington Business District, Mount Washington, Cincinnati, Hamilto...",39.095067,-84.386173,5.21,8.28


In [5]:
apt_sizes_df.head()

,Parcel Number,Name,Address,Sale Date,Sale Price,min_size,max_size
0,001-0001-0018-00,LDN INVESTMENTS LLC,6366 CORBLY RD,7/23/2019,"$340,000",4,19
1,001-0001-0307-00,SUN VALLEY PROPERTY LLC,6650 CORBLY RD,6/21/2019,NaN,4,19
2,001-0001-0359-00,MARKU IV PROPERTIES LLC,6401 COFFEY ST,4/25/2002,"$302,500",4,19
3,001-0001-0360-00,MARKU IV PROPERTIES LLC,6405 COFFEY ST,4/25/2002,"$302,500",4,19
4,001-0001-0361-00,JUMP ROBERT A SR & SUZANNE C,6409 COFFEY ST,12/31/1986,$0,4,19


In [6]:
apt_locations_df['COUNTYFP10'].value_counts()

61    6510
Name: COUNTYFP10, dtype: int64

# Add the min and max apartment sizes to the apt_locations_df dataframe. Only need to do this once per session. 

In [8]:
unique_apt_sizes_df = apt_sizes_df.drop_duplicates(subset=['Address'])

#apt_sizes_df
for index, row in apt_locations_df.iterrows():
    size_df = unique_apt_sizes_df[(unique_apt_sizes_df['Name'] == row['Name']) & (unique_apt_sizes_df['Address'] == row['Address'])]
    size_df = size_df.reset_index()
    if len(size_df) == 1:
        apt_locations_df.loc[index, 'min_size'] = size_df.loc[0, 'min_size']
        apt_locations_df.loc[index, 'max_size'] = size_df.loc[0, 'max_size']
    elif len(size_df) > 1:
        print("row ", index, " has ", len(size_df), " listings at ", row['Name'], ", ", row['Address'])

# Get neighborhood census tracts from here: 
https://www.cincinnati-oh.gov/planning/maps-and-data/census-demographics/

# Rename neighborhood accordingly

In [9]:
neighborhood = 'Walnut_Hills'

In [10]:
if neighborhood == 'EPH':
    tract_list = [92, 93, 94, 95, 96]
elif neighborhood == 'LPH':
    tract_list = [263]
elif neighborhood == 'WPH':
    tract_list = [97, 98, 99.01, 99.02, 100.05, 107, 212.02, 214.01]
elif neighborhood == 'Walnut_Hills':
    tract_list = [19, 36, 37, 267]
    
compare_list = []
for tract in tract_list:
    compare_list.append(int(100*tract))
    

In [11]:
compare_list

[1900, 3600, 3700, 26700]

In [12]:
neighborhood_dfs = {}

for tract in compare_list:
    neighborhood_dfs[tract] = apt_locations_df[apt_locations_df['TRACTCE10'] == tract]

In [13]:
#for tract in apt_locations_df['TRACTCE10']:
#    if str(tract).find('263') != -1:
#        print(tract)

for key in neighborhood_dfs:
    print(len(neighborhood_dfs[key]))

27
14
48
49


In [14]:
big_apts_df = pd.DataFrame(columns = apt_locations_df.columns.tolist())

for tract in neighborhood_dfs:
    #print(tract)
    tract_big_apts = neighborhood_dfs[tract][neighborhood_dfs[tract]['min_size'] == 40]
    big_apts_df = big_apts_df.append(tract_big_apts, ignore_index=True)

In [15]:
voter_file_df.head()

,SOS_VOTERID,COUNTY_NUMBER,COUNTY_ID,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,DATE_OF_BIRTH,REGISTRATION_DATE,VOTER_STATUS,PARTY_AFFILIATION,RESIDENTIAL_ADDRESS1,RESIDENTIAL_SECONDARY_ADDR,RESIDENTIAL_CITY,RESIDENTIAL_STATE,RESIDENTIAL_ZIP,RESIDENTIAL_ZIP_PLUS4,RESIDENTIAL_COUNTRY,RESIDENTIAL_POSTALCODE,MAILING_ADDRESS1,MAILING_SECONDARY_ADDRESS,MAILING_CITY,MAILING_STATE,MAILING_ZIP,MAILING_ZIP_PLUS4,MAILING_COUNTRY,MAILING_POSTAL_CODE,CAREER_CENTER,CITY,CITY_SCHOOL_DISTRICT,COUNTY_COURT_DISTRICT,CONGRESSIONAL_DISTRICT,COURT_OF_APPEALS,EDU_SERVICE_CENTER_DISTRICT,EXEMPTED_VILL_SCHOOL_DISTRICT,LIBRARY,LOCAL_SCHOOL_DISTRICT,MUNICIPAL_COURT_DISTRICT,PRECINCT_NAME,PRECINCT_CODE,STATE_BOARD_OF_EDUCATION,STATE_REPRESENTATIVE_DISTRICT,STATE_SENATE_DISTRICT,TOWNSHIP,VILLAGE,WARD,PRIMARY-03/07/2000,GENERAL-11/07/2000,SPECIAL-05/08/2001,GENERAL-11/06/2001,PRIMARY-05/07/2002,GENERAL-11/05/2002,SPECIAL-05/06/2003,GENERAL-11/04/2003,PRIMARY-03/02/2004,GENERAL-11/02/2004,SPECIAL-02/08/2005,PRIMARY-05/03/2005,PRIMARY-09/13/2005,GENERAL-11/08/2005,SPECIAL-02/07/2006,PRIMARY-05/02/2006,GENERAL-11/07/2006,PRIMARY-05/08/2007,PRIMARY-09/11/2007,GENERAL-11/06/2007,PRIMARY-11/06/2007,GENERAL-12/11/2007,PRIMARY-03/04/2008,PRIMARY-10/14/2008,GENERAL-11/04/2008,GENERAL-11/18/2008,PRIMARY-05/05/2009,PRIMARY-09/08/2009,PRIMARY-09/15/2009,PRIMARY-09/29/2009,GENERAL-11/03/2009,PRIMARY-05/04/2010,PRIMARY-07/13/2010,PRIMARY-09/07/2010,GENERAL-11/02/2010,PRIMARY-05/03/2011,PRIMARY-09/13/2011,GENERAL-11/08/2011,PRIMARY-03/06/2012,GENERAL-11/06/2012,PRIMARY-05/07/2013,PRIMARY-09/10/2013,PRIMARY-10/01/2013,GENERAL-11/05/2013,PRIMARY-05/06/2014,GENERAL-11/04/2014,PRIMARY-05/05/2015,PRIMARY-09/15/2015,GENERAL-11/03/2015,PRIMARY-03/15/2016,GENERAL-06/07/2016,PRIMARY-09/13/2016,GENERAL-11/08/2016,PRIMARY-05/02/2017,PRIMARY-09/12/2017,GENERAL-11/07/2017,PRIMARY-05/08/2018,GENERAL-08/07/2018,GENERAL-11/06/2018,PRIMARY-05/07/2019,PRIMARY-09/10/2019,GENERAL-11/05/2019,PRIMARY-03/17/2020
0,OH0019872779,31,1650735,SANSONE,MARY,L,NaN,1931-11-09,2008-07-29,ACTIVE,NaN,2277 BANNING RD,APT 111,CINCINNATI,OH,45239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CINCINNATI CITY,CINCINNATI PUBLIC SCHOOLS CITY SD,NaN,1,1,NaN,NaN,NaN,NaN,HAMILTON-CO,CINCINNATI 23-P,31-ANF,4,32,9,NaN,NaN,CINTI WARD 23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,X,NaN,NaN,X,NaN,NaN,NaN,NaN
1,OH0020953749,31,1702434,THOMPSON,JONNE,DENISE,NaN,1967-11-03,2010-01-11,ACTIVE,NaN,600 STEWART AVE,APT 18,CINCINNATI,OH,45215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GREAT OAKS CAREER CAMPUSES,NaN,NaN,NaN,2,1,HAMILTON COUNTY ESC,NaN,NaN,LOCKLAND LOCAL SD (HAMILTON),HAMILTON-CO,LOCKLAND B,31-BJM,4,33,9,NaN,LOCKLAND VILLAGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,X,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN
2,OH0018824688,31,1580672,WATERS,KRISTIN,Z,NaN,1967-11-16,2006-08-01,ACTIVE,D,7240 THUMBELINA LN,NaN,CINCINNATI,OH,45242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GREAT OAKS CAREER CAMPUSES,MONTGOMERY CITY,SYCAMORE COMMUNITY CITY SD,NaN,1,1,NaN,NaN,NaN,NaN,HAMILTON-CO,MONTGOMERY G,31-ATT,4,28,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,D,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN,NaN,X,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,X,R,NaN,NaN,X,NaN,NaN,X,NaN,NaN,X,NaN,NaN,X,D
3,OH0016287127,31,1548183,FERGUSON,WILLIAM,CHASE,NaN,1984-03-31,2005-01-10,ACTIVE,R,3415 WALLACE AVE,APT 28,CINCINNATI,OH,45226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CINCINNATI CITY,CINCINNATI PUBLIC SCHOOLS CITY SD,NaN,2,1,NaN,NaN,NaN,NaN,HAMILTON-CO,CINCINNATI 5-A,31-AEF,4,27,7,NaN,NaN,CINTI WARD 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [16]:
big_apts_df.head(3)

,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,index_right,Parcel Number,Name,Address,Sale Date,Sale Price,full_address,location,lat,lon,eviction-rate,eviction-filing-rate,min_size,max_size
0,39,61,3600,2,390610036002,Block Group 2,G5030,S,333896,0,39.135058,-84.489121,"POLYGON ((-84.48773299999999 39.13341399999999, -84.48775999999999 39.133367, -84.487792 39.1332...",846,060-0005-0012-00,KERPER DEVELOPMENT LTD,3066 KERPER AVE,3/9/1998,"$187,500","3066 KERPER AVE Hamilton County, Ohio","3066, Kerper Avenue, Peebles Corner, Walnut Hills, Cincinnati, Hamilton County, Ohio, 45206, Uni...",39.137258,-84.486753,2.20,4.40,40.0,1000000.0
1,39,61,3600,2,390610036002,Block Group 2,G5030,S,333896,0,39.135058,-84.489121,"POLYGON ((-84.48773299999999 39.13341399999999, -84.48775999999999 39.133367, -84.487792 39.1332...",972,065-0002-0101-00,WALNUT HILLS PRESERVATION LP,861 BEECHER ST,9/11/2015,$0,"861 BEECHER ST Hamilton County, Ohio","861, Beecher Street, Peebles Corner, Walnut Hills, Cincinnati, Hamilton County, Ohio, 45206, Uni...",39.133407,-84.489032,2.20,4.40,40.0,1000000.0
2,39,61,3700,1,390610037001,Block Group 1,G5030,S,509266,0,39.129242,-84.482503,"POLYGON ((-84.48424799999999 39.128165, -84.48423699999999 39.12880000000001, -84.48588599999999...",646,046-0006-0003-00,SEPTEM INVESTMENTS INC,2365 MADISON RD,7/15/1993,"$2,350,000","2365 MADISON RD Hamilton County, Ohio","Madison Road, Desales Corner Business District, East Walnut Hills, Cincinnati, Hamilton County, ...",39.129866,-84.478772,10.41,15.23,40.0,1000000.0


# V1 - output summary info on first sheet, then each subsequent sheet has breakdown of people in the apartment

In [20]:
residents_in_apt = {}

for apt_address in big_apts_df['Address'].unique():

    residents_df = voter_file_df[voter_file_df['RESIDENTIAL_ADDRESS1'] == apt_address]
    residents_df = residents_df.reset_index()
    residents_list = []

    for index, row in residents_df.iterrows():
        residents_list.append([row['LAST_NAME'], row['FIRST_NAME'], row['DATE_OF_BIRTH'], row['RESIDENTIAL_ADDRESS1'], row['RESIDENTIAL_SECONDARY_ADDR']])

    residents_in_apt[apt_address] = pd.DataFrame(residents_list, columns = ['last_name', 'first_name', 'dob', 'address1', 'address2'])

In [21]:
big_apts_to_excel = big_apts_df[['STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'BLKGRPCE10', 'GEOID10', 'Parcel Number', 'Name', 'Sale Price', 'location', 'eviction-rate', 'eviction-filing-rate', 'min_size', 'max_size']]

version = 1

#file = r'C:\Users\kevin\Documents\jupyter notebooks\evictions\Walnut Hills Apts\\' + neighborhood + ' v' + str(version) + '.csv'

with pd.ExcelWriter(r'evictions\Walnut Hills Apts\\' + neighborhood + ' v' + str(version) + '.xlsx') as writer:  
    big_apts_to_excel.to_excel(writer, sheet_name='Summary_Sheet', index=False)
    for key in residents_in_apt:
        residents_in_apt[key].to_excel(writer, sheet_name = key, index=False)